# Introduction to Software Profiling

#### What is profiling?

Profiling is the analysis of how the code behaves in relation to the resources it's using. It can be in relation to space (memory) or time complexity of a program, the usage of particular instructions, or the frequency and duration of function calls.

- Algorithmic complexity
- Profilers for Python
- The ecossystem
- Tips & Techniques
- (OPT //- Difference between Multiprocessing, multithreading, async io (MAYBE) (read chap 19 from Luciano's)

## Algorithmic complexity


In [ ]:
def foo(i):
    answer = 1
    while i >= 1:
        answer *= i
        i -= 1
    return answer

👥 What are some ways you would try to measure how long this function would take to run?

In [ ]:
# using time
import datetime

tstart = None
tend = None

def start_time():
    global tstart
    tstart = datetime.datetime.now()

def get_delta():
    global tstart
    tend = datetime.datetime.now()
    return tend - tstart

In [ ]:
start_time()
print('start foo')
delta1 = get_detal()

start_time()
fooTime = foo(100)
delta2 = get_delta()

Let's use a random accesss machine as our model with sequential steps.

In [ ]:
def linearSearch(List, val):
    for element in List:
        if element == val:
            return True

👥 What happens to this function run-time if my array is:
- a) Very large
- b) Very small
- c) Ordered
- d) Unordered

### Asymptotic notation

Used to classify algorithms according to how their run time or space requirements grow as the input size grows.

In [ ]:
def foo(x):
    ans = 0
    for i in range (1000):
        ans += 1
    for i in range(x):
        ans += 1
    for i in range(x):
        for j in range(x):
            ans += 1
            ans += 1

This function's steps iterations be described as:

1000 + x + 2x²

The most commonly used asymptotic notation is called Big O, it's used to give an upper bound on the asymptotic growth of a function.

For example: O(n²) means the function grows no faster than the quadratic polynomial n².

The most common complexity classes:

- O(1) constant running time
- O(log n) logarithmic running time
- O(n) linear running time
- O(n^k) polynomial  running time
- O(c^n) exponential running time. n = a power based on the size of the input

**Constant**

Several [Python operations](https://wiki.python.org/moin/TimeComplexity) are constant!

**Logarithmic**

Here we don't care if we're using different bases because their difference is merely a constant multiplicative factor.

In [ ]:
def intToStr(i):
    digits = '0123456789'
    if i == 0:
        return '0'
    result = ''
    while i > 0:
        result = digits[i % 10] + result
        i = i/10
    return result

👥 Can you tell why the next function is O(log n)?

In [ ]:
def addDigits(n):
    stringRep = intToStr(n)
    val = 0
    for c in stringRep:
        val += int(c)
    return val

The complexity of converting 𝑛 to a string is 𝑂(log𝑛), and intToStr returns a string of length 𝑂(log𝑛). The for loop will be executed 𝑂(len(stringRep)) times, i.e., 𝑂(log𝑛) times. Putting it all together, and assuming that a character representing a digit can be converted to an integer in constant time, the program will run in time proportional to 𝑂(log𝑛)+𝑂(log𝑛), which makes it 𝑂(log𝑛).

**Exponential complexity**

Often recursive algorithms that solve a problem of size N by recursively solving smaller problems of size N-1.

- Breaking a password

### Built-in Jupyter methods

In [8]:
import pandas as pd
import numpy as np


df = pd.DataFrame({
        "a": np.random.randn(1000),
        "b": np.random.randn(1000),
        "N": np.random.randint(100, 1000, (1000)),
        "x": "x",
    })
df

,a,b,N,x
0,-0.768250,1.290252,342,x
1,0.930606,0.606400,270,x
2,0.058078,-0.324894,596,x
3,0.318598,-1.343317,403,x
4,-0.460761,1.472222,123,x
...,...,...,...,...
995,-0.833435,-0.570195,558,x
996,-0.356861,0.604569,508,x
997,0.186917,-1.859684,602,x
998,-0.721102,0.462842,285,x


In [9]:
def f(x):
    return x * (x - 1)

def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N

    for i in range(N):
        s += f(a + i * dx)

    return s * dx

In [10]:
%timeit df.apply(lambda x: integrate_f(x["a"], x["b"], x["N"]), axis=1)

58.6 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%prun -l 4 df.apply(lambda x: integrate_f(x["a"], x["b"], x["N"]), axis=1)  # noqa E999

         638134 function calls (638116 primitive calls) in 0.120 seconds

   Ordered by: internal time
   List reduced from 222 to 4 due to restriction <4>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1000    0.066    0.000    0.096    0.000 4281555139.py:4(integrate_f)
   552216    0.030    0.000    0.030    0.000 4281555139.py:1(f)
     3000    0.003    0.000    0.014    0.000 series.py:943(__getitem__)
     1003    0.002    0.000    0.003    0.000 series.py:640(name)

### Tracing

`sys.setprofile` sets a trace function that's triggered every time the VM enters or exits both Python and C functions.

In [ ]:
# ⚠️ Run this code on your Python CLI

# def fib(n):
#     i, f1, f2 = 1, 1, 1
#     while i < n:
#         f1, f2 = f2, f1 + f2
#         i += 1
#     return f1

# import opcode


# def show_trace(frame, event, arg):
#     frame.f_trace_opcodes = True
#     code = frame.f_code
#     offset = frame.f_lasti

#     print(f"| {event:10} | {str(arg):>4} |", end=' ')
#     print(f"{frame.f_lineno:>4} | {frame.f_lasti:>6} |", end=' ')
#     print(f"{opcode.opname[code.co_code[offset]]:<18} | {str(frame.f_locals):<35} |")
#     return show_trace

# import sys

# header = f"| {'event':10} | {'arg':>4} | line | offset | {'opcode':^18} | {'locals':^35} |"
# print(header)
# sys.settrace(show_trace)
# fib(3)
# sys.settrace(None)

### Deterministic profiling

#### cProfile

- Offers you both the total running time of cProfile.run(statement, filename=None, sort=-1)a software as well as specific function calls and times
- Nice integration with GUI tools and pstats

In [ ]:
import cProfile

# cProfile.run(statement, filename=None, sort=-1)

You can pass python code or a function name that you want to profile as a string to the statement argument.

In [ ]:
import numpy as np

cProfile.run("2**200000")

- ncalls : Shows the number of calls made
- tottime: Total time taken by the given function. Note that the time made in calls to sub-functions are excluded.
- percall: Total time / No of calls. ( remainder is left out )
- cumtime: Unlike tottime, this includes time spent in this and all subfunctions that the higher-level function calls. It is most useful and is accurate for recursive functions.
- The percall following cumtime is calculated as the quotient of cumtime divided by primitive calls. The primitive calls include all the calls that were not included through recursion.

In [ ]:
def add_emoji():
    arr=[]
    arr.append('🔥')

def multiply():
    arr=[]
    for i in range(0,400000):
        arr.append(i * 2)
        add_emoji()

def main():
    multiply()
    print('end')

if __name__ == '__main__':
    cProfile.run('main()')

You can save the data using the following:

In [ ]:
import cProfile, pstats
profiler = cProfile.Profile()
stats = pstats.Stats(profiler)
stats.dump_stats('/content/export-data')

In [ ]:
And use a GUI to visualize it called snakeviz:

In [ ]:
# installing the module
!pip install snakeviz

In [ ]:
# load it on the notebook
%load_ext snakeviz <filename>

In [ ]:
# opens snakeviz
%snakeviz main()

cProfile has a lot more to offer and I recommend checking the [Python docs](https://docs.python.org/3/library/profile.html#module-cProfile) to learn more about its specific functions.

### Statistical profiling

Samples the program counter at regular intervals. The numbers will be statistical approximations instead of exact numbers because of the several process in place.

- Less data to analyze
- Smaller profiling footprint

#### pprofile

pprofile offers both deterministic and statistical modes for profiling. We're going to take a look in the statistical mode:

In [ ]:
# import threading
# import time


# def func():
#     time.sleep(1)

# def func2():
#     pass

# t1 = threading.Thread(target=func)
# t2 = threading.Thread(target=func)
# t1.start()
# t2.start()
# (func(), func2())
# t1.join()
# t2.join()

In statistic profiling mode, pprofile periodically snapshots the current callstack(s) of current process to see what is being executed. As a result, profiler overhead can be dramatically reduced, making it possible to profile real workloads. But, of course, the information provided by this method is less precise.

In [1]:
# Fluent Python's implementation of a time function
import time

def clock(func):
    def clocked(*args):
        t0 = time.time()
        result = func(*args)
        elapsed = time.time() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [2]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

print(fibonacci(4))

[0.00000119s] fibonacci(0) -> 0
[0.00000191s] fibonacci(1) -> 1
[0.00314498s] fibonacci(2) -> 1
[0.00000095s] fibonacci(1) -> 1
[0.00000119s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00002289s] fibonacci(2) -> 1
[0.00004292s] fibonacci(3) -> 2
[0.00321102s] fibonacci(4) -> 3
3


In [3]:
import functools 

@functools.cache
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

print(fibonacci(4))

[0.00000000s] fibonacci(0) -> 0
[0.00000095s] fibonacci(1) -> 1
[0.00005412s] fibonacci(2) -> 1
[0.00000095s] fibonacci(3) -> 2
[0.00006294s] fibonacci(4) -> 3
3


All the arguments taken by the decorated function must be hashable, because the underlying lru_cache uses a dict to store the results

`@cache` won't be available to you if your Python < 3.8, but you can still use `@lru_cache`.

`@lru_cache` contains two arguments, `maxsize` which receives an integer containing the maximum number of entries to be stored and `typed` which receives a boolean that will say if arguments ought to be stored separately.